# 1.Import and Install Modules

In [ ]:
!pip install simpletransformers
!pip install transformers
!pip install music21

In [ ]:
import music21
import time
from music21 import *
from tqdm.notebook import tqdm, trange
import pandas as pd
import logging
import glob
import string
from sklearn.model_selection import train_test_split as tts
import logging

# 2. Process the Midi files to text like format

In [ ]:
files = glob.glob('/kaggle/input/smaller-midi/MIDI Truncated/*.midi')

training_notes = []
training_duration = []

for file in tqdm(files):
        notes = []
        durations = []

        #parsing MIDI files one by one
        original_score = music21.converter.parse(file).chordify()

        #depending on the element found in the instrument. Like we would have 'rest' for drums,
        #chords for guitar
        for element in original_score.flat:
            note_name = None
            duration_name  = None

            #metadata  
            if isinstance(element, music21.key.Key):
                note_name = str(element.tonic.name) + ':' + str(element.mode)
                duration_name = "0.0"
            
            #metadata
            elif isinstance(element, music21.meter.TimeSignature):
                note_name = str(element.ratioString) + 'TS'
                duration_name = "0.0"

            elif isinstance(element, music21.chord.Chord):
                note_name = element.pitches[-1].nameWithOctave
                duration_name = str(element.duration.quarterLength)

            # As using drums data, elements found would be 'rest'
            elif isinstance(element, music21.note.Rest):
                note_name = str(element.name)
                duration_name = str(element.duration.quarterLength)

            elif isinstance(element, music21.note.Note):
                note_name = str(element.nameWithOctave)
                duration_name = str(element.duration.quarterLength)

            if note_name and duration_name:
                notes.append(note_name)
                durations.append(duration_name)
        
        #notes and duration hold seuence for one music piece. 
        training_notes.append(notes)
        training_duration.append(durations)

In [ ]:
print(training_notes[1])
print(training_duration[1])

# Sliding Window Implentation

In [ ]:
train = []
label = []
window_size = 10
for x,y in zip(training_notes,training_duration):
    
    if len(x)>window_size:
            for index in range(len(x)-1-window_size):
                in_1 = x[index:index+window_size] + ['NA'] + y[index:index+window_size]
                out_1 = x[index+1:index+1+window_size] + ['NA'] + y[index+1:index+1+window_size]
                train.append(in_1)
                label.append(out_1)
    else:
        pass

# Tokenising Data

In [ ]:
train_df = pd.DataFrame({'input_text':[x for x in train],'target_text':[x for x in label]})
print(train_df['input_text'].iloc[:10])
map_dict = {}
unique_values = sorted(train_df['input_text'].explode().unique())
for index, value in enumerate(unique_values):
    index = index%26
    map_dict[value] = chr(index + 65)
    print(index)

#map_dict = {y:string.ascii_uppercase[x] for x,y in enumerate(sorted(train_df['input_text'].explode().unique()))}

train_df = train_df.applymap(lambda x: ' '.join([map_dict[y] for y in x]))

training, validation = tts(train_df)
training = training.sample(len(training))

****The logic followed is quite simple****

Create a DataFrame using the input and output sequences generated above.

Get all unique tokens used (notes+duration both combined) and assign ASCII characters for each unique token’s representation.

Replace tokens used with the new mapping and form a single string separated by spaces.

Do a train-test split and shuffling.

****Let’s observe the training data ****

In [ ]:
training

# 3. Feed the data into the pretrained Transformer

In [ ]:
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

#model args to be used
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 41,
    "train_batch_size": 10,
    "num_train_epochs": 1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "manual_seed": 4,
}

encoder_type = "roberta"


#The 1st 3 parameters are model type, encoder and decoder exact model name to be used
model = Seq2SeqModel(
    encoder_type,
    "roberta-base",
    "bert-base-cased",
    args=model_args,
    use_cuda=True,
)

# 4. Training

In [ ]:
results = model.train_model(training)

In [ ]:
results = model.eval_model(validation)

In [ ]:
results